<a href="https://colab.research.google.com/github/vaniamv/dataprocessing/blob/main/spark_streaming/challenges/final_challenges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up PySpark

In [ ]:
%pip install pyspark

# Context
Message events are coming from platform message broker (kafka, pubsub, kinesis...).
You need to process the data according to the requirements.

Message schema:
- timestamp
- value
- event_type
- message_id
- country_id
- user_id



# Challenge 1

Step 1
- Change exising producer
	- Change parquet location to "/content/lake/bronze/messages/data"
	- Add checkpoint (/content/lake/bronze/messages/checkpoint)
	- Delete /content/lake/bronze/messages and reprocess data
	- For reprocessing, run the streaming for at least 1 minute, then stop it

Step 2
- Implement new stream job to read from messages in bronze layer and split result in two locations
	- "messages_corrupted"
		- logic: event_status is null, empty or equal to "NONE"
		- extra logic: add country name by joining message with countries dataset
		- partition by "date" -extract it from timestamp
		- location: /content/lake/silver/messages_corrupted/data

	- "messages"
		- logic: not corrupted data
		- extra logic: add country name by joining message with countries dataset
		- partition by "date" -extract it from timestamp
		- location: /content/lake/silver/messages/data

	- technical requirements
		- add checkpint (choose location)
		- use StructSchema
		- Set trigger interval to 5 seconds
		- run streaming for at least 20 seconds, then stop it

	- alternatives
		- implementing single streaming job with foreach/- foreachBatch logic to write into two locations
		- implementing two streaming jobs, one for messages and another for messages_corrupted
		- (paying attention on the paths and checkpoints)


  - Check results:
    - results from messages in bronze layer should match with the sum of messages+messages_corrupted in the silver layer

In [1]:
%pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.8 MB/s eta 0:00:00


# Producer

In [70]:
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from faker import Faker
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Test streaming').getOrCreate()
sc = spark.sparkContext

fake = Faker()
messages = [fake.uuid4() for _ in range(50)]

def enrich_data(df, messages=messages):
  fake = Faker()
  new_columns = {
      'event_type': F.lit(fake.random_element(elements=('OPEN', 'RECEIVED', 'SENT', 'CREATED', 'CLICKED', '', 'NONE'))),
      'message_id': F.lit(fake.random_element(elements=messages)),
      'channel': F.lit(fake.random_element(elements=('CHAT', 'EMAIL', 'SMS', 'PUSH', 'OTHER'))),
      'country_id': F.lit(fake.random_int(min=2000, max=2015)),
      'user_id': F.lit(fake.random_int(min=1000, max=1050)),
  }
  df = df.withColumns(new_columns)
  return df

def insert_messages(df: DataFrame, batch_id):
  enrich = enrich_data(df)
  enrich.write.mode("append").format("parquet").save("content/lake/bronze/messages/data")

# read stream
df_stream = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

# write stream
query = (df_stream.writeStream
.outputMode('append')
.option('checkpointLocation', 'content/lake/bronze/checkpoint')
.trigger(processingTime='1 seconds')
.foreachBatch(insert_messages)
.start()
)

query.awaitTermination(20)


False

In [34]:
query.isActive

True

In [130]:
query.stop()

In [69]:
!rm -rf content

In [71]:
df = spark.read.format("parquet").load("content/lake/bronze/messages/*")
df.show()

+--------------------+-----+----------+--------------------+-------+----------+-------+
|           timestamp|value|event_type|          message_id|channel|country_id|user_id|
+--------------------+-----+----------+--------------------+-------+----------+-------+
|2024-12-10 23:09:...|   15|  RECEIVED|25ab32a2-e74a-4a5...|  EMAIL|      2012|   1044|
|2024-12-10 23:08:...|    3|  RECEIVED|ecf75ee4-03d7-442...|  OTHER|      2015|   1002|
|2024-12-10 23:09:...|    9|  RECEIVED|aaa3115b-782e-43c...|  OTHER|      2012|   1002|
|2024-12-10 23:09:...|    7|  RECEIVED|cdd30d8c-50a9-4e8...|   PUSH|      2000|   1017|
|2024-12-10 23:09:...|    8|   CLICKED|df20615e-bd7d-497...|   CHAT|      2015|   1045|
|2024-12-10 23:09:...|   17|   CREATED|78589f69-7993-41e...|   CHAT|      2002|   1010|
|2024-12-10 23:09:...|   12|   CLICKED|89ccfcd0-1435-455...|    SMS|      2012|   1035|
|2024-12-10 23:09:...|   13|   CREATED|718e7a23-2bb0-416...|    SMS|      2002|   1025|
|2024-12-10 23:09:...|   18|   C

In [82]:
df.schema

StructType([StructField('timestamp', TimestampType(), True), StructField('value', LongType(), True), StructField('event_type', StringType(), True), StructField('message_id', StringType(), True), StructField('channel', StringType(), True), StructField('country_id', IntegerType(), True), StructField('user_id', IntegerType(), True)])

In [72]:
df.count()

19

# Additional datasets

In [73]:
countries = [
    {"country_id": 2000, "country": "Brazil"},
    {"country_id": 2001, "country": "Portugal"},
    {"country_id": 2002, "country": "Spain"},
    {"country_id": 2003, "country": "Germany"},
    {"country_id": 2004, "country": "France"},
    {"country_id": 2005, "country": "Italy"},
    {"country_id": 2006, "country": "United Kingdom"},
    {"country_id": 2007, "country": "United States"},
    {"country_id": 2008, "country": "Canada"},
    {"country_id": 2009, "country": "Australia"},
    {"country_id": 2010, "country": "Japan"},
    {"country_id": 2011, "country": "China"},
    {"country_id": 2012, "country": "India"},
    {"country_id": 2013, "country": "South Korea"},
    {"country_id": 2014, "country": "Russia"},
    {"country_id": 2015, "country": "Argentina"}
]

countries = spark.createDataFrame(countries)

# Streaming Messages x Messages Corrupted

In [ ]:
df.select('event_type').distinct().collect()


[Row(event_type='CLICKED'),
 Row(event_type='CREATED'),
 Row(event_type='RECEIVED'),
 Row(event_type='NONE'),
 Row(event_type=''),
 Row(event_type='OPEN'),
 Row(event_type='SENT')]

In [116]:
!rm -rf content/lake/silver

In [108]:
# TODO
from pyspark.sql.functions import col, lit

corrupted_condition = (col("event_type").isNull() |
                       (col("event_type") == "") |
                       (col("event_type") == "NONE"))

In [109]:
from pyspark.sql.types import StructType, StructField, StringType, LongType, IntegerType, TimestampType


In [123]:
def insert_messages_silver(df: DataFrame, batch_id):
  corrupted_df = df.filter(corrupted_condition)
  corrupted_df = corrupted_df.join(countries, on="country_id", how="left")
  corrupted_df.write.mode("append").format("parquet").partitionBy("date").save("content/lake/silver/messages_corrupted/data")
  non_corrupted_df = df.filter(~corrupted_condition)
  non_corrupted_df = non_corrupted_df.join(countries, on="country_id", how="left")
  non_corrupted_df.write.mode("append").format("parquet").partitionBy("date").save("content/lake/silver/messages/data")

schema = StructType([
    StructField('timestamp', TimestampType(), True),
    StructField('value', LongType(), True),
    StructField('event_type', StringType(), True),
    StructField('message_id', StringType(), True),
    StructField('channel', StringType(), True),
    StructField('country_id', IntegerType(), True),
    StructField('user_id', IntegerType(), True)
    ])

# Read the streaming data with the specified schema
messages_df = spark.readStream.format("parquet").schema(schema).load("content/lake/bronze/messages/data")

#extract date from timestamp
messages_df = messages_df.withColumn("date", F.to_date("timestamp"))

query_1 = (messages_df.writeStream
.outputMode('append')
.option('checkpointLocation', 'content/lake/silver/checkpoint')
.trigger(processingTime='1 seconds')
.foreachBatch(insert_messages_silver)
.start()
)


In [128]:
query_1.stop()

ERROR:py4j.clientserver:There was an exception while executing the Python Proxy on the Python Side.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 617, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/utils.py", line 120, in call
    raise e
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/utils.py", line 117, in call
    self.func(DataFrame(jdf, wrapped_session_jdf), batch_id)
  File "<ipython-input-123-d746a393f72d>", line 7, in insert_messages_silver
    non_corrupted_df.write.mode("append").format("parquet").partitionBy("date").save("content/lake/silver/messages/data")
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/readwriter.py", line 1463, in save
    self._jwrite.save(path)
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
  

In [124]:
query_1.isActive

True

In [127]:
df_silver = spark.read.format("parquet").load("content/lake/silver/messages/data/*")
df_silver.show()

+----------+--------------------+-----+----------+--------------------+-------+-------+-------+
|country_id|           timestamp|value|event_type|          message_id|channel|user_id|country|
+----------+--------------------+-----+----------+--------------------+-------+-------+-------+
|      2000|2024-12-10 23:20:...|  681|      SENT|7100ee58-2e9f-47e...|  OTHER|   1000| Brazil|
|      2000|2024-12-10 23:20:...|  683|      SENT|7100ee58-2e9f-47e...|  OTHER|   1000| Brazil|
|      2000|2024-12-10 23:24:...|  912|      OPEN|de218c35-8ef3-45e...|   PUSH|   1001| Brazil|
|      2000|2024-12-10 23:24:...|  914|      OPEN|de218c35-8ef3-45e...|   PUSH|   1001| Brazil|
|      2000|2024-12-10 23:14:...|  360|  RECEIVED|75afef40-6d71-455...|  EMAIL|   1022| Brazil|
|      2000|2024-12-10 23:27:...| 1124|  RECEIVED|3bb8685f-dddf-42f...|  EMAIL|   1044| Brazil|
|      2000|2024-12-10 23:27:...| 1128|  RECEIVED|155cef0b-4c75-45d...|  EMAIL|   1007| Brazil|
|      2000|2024-12-10 23:28:...| 1169| 

In [126]:
df_silver_corrupted = spark.read.format("parquet").load("content/lake/silver/messages_corrupted/data/*")
df_silver_corrupted.show()

+----------+--------------------+-----+----------+--------------------+-------+-------+-------+
|country_id|           timestamp|value|event_type|          message_id|channel|user_id|country|
+----------+--------------------+-----+----------+--------------------+-------+-------+-------+
|      2000|2024-12-10 23:09:...|   25|      NONE|4045ec8c-f9c1-495...|  EMAIL|   1033| Brazil|
|      2000|2024-12-10 23:20:...|  723|      NONE|de218c35-8ef3-45e...|  EMAIL|   1029| Brazil|
|      2000|2024-12-10 23:25:...|  969|      NONE|a0b9524b-741a-41f...|  EMAIL|   1043| Brazil|
|      2000|2024-12-10 23:13:...|  266|      NONE|f7527a10-0c04-4f5...|  EMAIL|   1050| Brazil|
|      2000|2024-12-10 23:14:...|  334|      NONE|33d15c55-12a8-45c...|  OTHER|   1011| Brazil|
|      2000|2024-12-10 23:27:...| 1141|      NONE|78589f69-7993-41e...|  OTHER|   1026| Brazil|
|      2000|2024-12-10 23:09:...|   16|      NONE|0fcd5fed-13d8-454...|  OTHER|   1012| Brazil|
|      2000|2024-12-10 23:17:...|  529| 

## Checking data

In [ ]:
# TODO

# Challenge 2

- Run business report
- But first, there is a bug in the system which is causing some duplicated messages, we need to exclude these lines from the report

- removing duplicates logic:
  - Identify possible duplicates on message_id, event_type and channel
  - in case of duplicates, consider only the first message (occurrence by timestamp)
  - Ex:
    In table below, the correct message to consider is the second line

```
    message_id | channel | event_type | timestamp
    123        | CHAT    | CREATED    | 10:10:01
    123        | CHAT    | CREATED    | 07:56:45 (first occurrence)
    123        | CHAT    | CREATED    | 08:13:33
```

- After cleaning the data we're able to create the busines report

In [ ]:
# dedup data
from pyspark.sql import functions as F
from pyspark.sql.window import Window
df = spark.read.format("parquet").load("content/lake/silver/messages")
dedup = df.withColumn("row_number", F.row_number().over(Window.partitionBy("message_id", "event_type", "channel").orderBy("timestamp"))).filter("row_number = 1").drop("row_number")

### Report 1
  - Aggregate data by date, event_type and channel
  - Count number of messages
  - pivot event_type from rows into columns
  - schema expected:
  
```
|      date|channel|CLICKED|CREATED|OPEN|RECEIVED|SENT|
+----------+-------+-------+-------+----+--------+----+
|2024-12-03|    SMS|      4|      4|   1|       1|   5|
|2024-12-03|   CHAT|      3|      7|   5|       8|   4|
|2024-12-03|   PUSH|   NULL|      3|   4|       3|   4|
```

In [ ]:
# report 1
# TODO

## Report 2

- Identify the most active users by channel (sorted by number of iterations)
- schema expected:

```
+-------+----------+----+-----+-----+----+---+
|user_id|iterations|CHAT|EMAIL|OTHER|PUSH|SMS|
+-------+----------+----+-----+-----+----+---+
|   1022|         5|   2|    0|    1|   0|  2|
|   1004|         4|   1|    1|    1|   1|  0|
|   1013|         4|   0|    0|    2|   1|  1|
|   1020|         4|   2|    0|    1|   1|  0|
```


In [ ]:
# report 2
# TODO

# Challenge 3

In [ ]:
# Theoretical question:

# A new usecase requires the message data to be aggregate in near real time
# They want to build a dashboard embedded in the platform website to analyze message data in low latency (few minutes)
# This application will access directly the data aggregated by streaming process

# Q1:
- What would be your suggestion to achieve that using Spark Structure Streaming?
Or would you choose a different data processing tool?

- Which storage would you use and why? (database?, data lake?, kafka?)

